### Loading Libraries & Environment Variables

In [18]:
import os
import requests
import json
import prettytable
import pandas as pd
from dotenv import load_dotenv

In [19]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

### Loading Data

#### Setting up to use the BLS API

In [20]:
# Set the base url for BLS API v2
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

In [21]:
# Pull the API key in .env which contains: bls_key = 'key'
bls_key = os.getenv("bls_key")
key = '?registrationkey={}'.format(bls_key)

In [22]:
# Set the start year and end year for the 5-year data series
dates = ('2017', '2021')

#### Loading Labor Force data for Washington Counties

In [23]:
# Creating series list
# example: 'LAUCN530010000000005'
# The series id has the following structure:
# 'LA' = Prefix for local area data
# 'U' (or 'S') for seasonal adjustment 
# 'CN53001000000' = area code --> 'CN' for county level data + state fips code + county fips code
# '06' = 2-digit measure code for Labor Force

# Create a dictionary of the series codes and fips codes
series_dict = {
    'LAUCN530110000000006': '53011',
    'LAUCN530250000000006': '53025',
    'LAUCN530330000000006': '53033',
    'LAUCN530350000000006': '53035',
    'LAUCN530530000000006': '53053',
    'LAUCN530610000000006': '53061',
    'LAUCN530650000000006': '53065',
    'LAUCN530670000000006': '53067',
    }

# The following code is modeled from an example provided at https://www.bd-econ.com/blsapi.html.

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})


In [24]:
# Posting request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [25]:
# Creating a date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Creating an empty dataframe to fill with values
df_labor_force = pd.DataFrame()

# Building a pandas series from the API results, p
for s in p:
    df_labor_force[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]


In [26]:
#Reviewing labor force dataframe
df_labor_force.head()

,53011,53025,53033,53035,53053,53061,53065,53067
2017-01-01,226512.0,41475.0,1221410.0,120338.0,416321.0,417997.0,18119.0,134112.0
2017-02-01,227623.0,42406.0,1233614.0,119754.0,414814.0,422154.0,18150.0,133964.0
2017-03-01,227629.0,43573.0,1234491.0,119692.0,415076.0,422869.0,18223.0,134407.0
2017-04-01,226539.0,43944.0,1231264.0,119293.0,412064.0,421228.0,18108.0,133252.0
2017-05-01,225928.0,44395.0,1233583.0,119549.0,412203.0,422160.0,18038.0,132929.0


In [27]:
df_labor_force.tail()

,53011,53025,53033,53035,53053,53061,53065,53067
2021-07-01,241162.0,49358.0,1316199.0,126130.0,442907.0,450952.0,19031.0,144040.0
2021-08-01,242757.0,47657.0,1317920.0,125290.0,444189.0,451339.0,18787.0,144346.0
2021-09-01,241084.0,49628.0,1309988.0,125812.0,443144.0,448365.0,18863.0,144507.0
2021-10-01,244095.0,49056.0,1298675.0,127212.0,442830.0,444486.0,19032.0,145657.0
2021-11-01,248353.0,47842.0,1287575.0,128960.0,448188.0,439694.0,19264.0,147748.0


In [28]:
df_labor_force.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 59 entries, 2017-01-01 to 2021-11-01
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   53011   59 non-null     float64
 1   53025   59 non-null     float64
 2   53033   59 non-null     float64
 3   53035   59 non-null     float64
 4   53053   59 non-null     float64
 5   53061   59 non-null     float64
 6   53065   59 non-null     float64
 7   53067   59 non-null     float64
dtypes: float64(8)
memory usage: 4.1 KB


In [29]:
#Saving the first file to a csv
df_labor_force.to_csv('Resources/wa_labor_force_by_county.csv')

### Loading Employment data for Washington Counties

In [31]:
# Creating series list
# example: 'LAUCN530010000000005'
# The series id has the following structure:
# 'LA' = Prefix for local area data
# 'U' (or 'S') for seasonal adjustment 
# 'CN53001000000' = area code --> 'CN' for county level data + state fips code + county fips code
# '05' = 2-digit measure code for Employment

# Creating a dictionary of the series codes and fips codes
# Create a dictionary of the series codes and fips codes
series_dict = {
    'LAUCN530110000000005': '53011',
    'LAUCN530250000000005': '53025',
    'LAUCN530330000000005': '53033',
    'LAUCN530350000000005': '53035',
    'LAUCN530530000000005': '53053',
    'LAUCN530610000000005': '53061',
    'LAUCN530650000000005': '53065',
    'LAUCN530670000000005': '53067',
    }
# The following code is modeled from an example provided at https://www.bd-econ.com/blsapi.html.

# Specifying json as content type to return
headers = {'Content-type': 'application/json'}

# Submitting the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})


In [32]:
# Posting request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [33]:
# Creating a date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Creating an empty dataframe to fill with values for employment
df_employment = pd.DataFrame()

# Building a pandas series from the API results, p
for s in p:
    df_employment[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]

In [34]:
df_employment.head()

,53011,53025,53033,53035,53053,53061,53065,53067
2017-01-01,213014.0,37313.0,1175970.0,113713.0,391405.0,400624.0,16460.0,126521.0
2017-02-01,214725.0,38473.0,1190267.0,113417.0,390557.0,405531.0,16488.0,126577.0
2017-03-01,215734.0,40261.0,1193174.0,113806.0,392219.0,406766.0,16580.0,127585.0
2017-04-01,215649.0,41421.0,1194747.0,113773.0,391046.0,407298.0,16746.0,127038.0
2017-05-01,215336.0,42038.0,1192269.0,114114.0,391487.0,406513.0,16853.0,126845.0


In [35]:
df_employment.tail()

,53011,53025,53033,53035,53053,53061,53065,53067
2021-07-01,230130.0,47026.0,1249573.0,120874.0,420492.0,424164.0,18081.0,137895.0
2021-08-01,231062.0,45169.0,1255212.0,119704.0,420777.0,426219.0,17726.0,137862.0
2021-09-01,231348.0,47547.0,1252227.0,121150.0,423146.0,424829.0,17981.0,139109.0
2021-10-01,234164.0,46922.0,1241423.0,122368.0,422714.0,421286.0,18157.0,140085.0
2021-11-01,240028.0,45715.0,1250054.0,125055.0,431005.0,424025.0,18451.0,143056.0


In [36]:
df_employment.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 59 entries, 2017-01-01 to 2021-11-01
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   53011   59 non-null     float64
 1   53025   59 non-null     float64
 2   53033   59 non-null     float64
 3   53035   59 non-null     float64
 4   53053   59 non-null     float64
 5   53061   59 non-null     float64
 6   53065   59 non-null     float64
 7   53067   59 non-null     float64
dtypes: float64(8)
memory usage: 4.1 KB


In [38]:
#Saving the first file to csv
df_employment.to_csv('Resources/wa_employment_by_county.csv')